In [ ]:
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt

--2023-12-28 15:57:43--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228391 (223K) [text/plain]
Saving to: ‘train_aspects.txt.3’

train_aspects.txt.3 100%[===================>] 223.04K  --.-KB/s    in 0.01s   

2023-12-28 15:57:43 (15.7 MB/s) - ‘train_aspects.txt.3’ saved [228391/228391]

--2023-12-28 15:57:43--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting re

In [ ]:
! pip install transformers -U

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# Загрузка данных

In [ ]:
import pandas as pd

In [ ]:
texts = pd.read_csv('train_reviews.txt', delimiter='\t', names = ['review_id', 'text'])

In [ ]:
train_aspects = pd.read_csv('train_aspects.txt', delimiter='\t', names = ['review_id', 'category', 'mention', 'start', 'stop', 'sentiment'])

In [ ]:
train_aspects['sentiment'].value_counts()

positive    3104
neutral      910
negative     660
both          89
Name: sentiment, dtype: int64

In [ ]:
data = pd.merge(texts, train_aspects, on='review_id')

In [ ]:
data

,review_id,text,category,mention,start,stop,sentiment
0,3976,"День 8-го марта прошёл, можно и итоги подвести...",Whole,ресторане,71,80,neutral
1,3976,"День 8-го марта прошёл, можно и итоги подвести...",Whole,ресторанах,198,208,neutral
2,3976,"День 8-го марта прошёл, можно и итоги подвести...",Whole,ресторане,256,265,neutral
3,3976,"День 8-го марта прошёл, можно и итоги подвести...",Service,Столик бронировали,267,285,neutral
4,3976,"День 8-го марта прошёл, можно и итоги подвести...",Service,администратор,322,335,positive
...,...,...,...,...,...,...,...
4758,16630,Уютная и тёплая домашняя обстановка! Милый и о...,Service,обслуживание,85,97,positive
4759,16630,Уютная и тёплая домашняя обстановка! Милый и о...,Food,Еда,99,102,positive
4760,16630,Уютная и тёплая домашняя обстановка! Милый и о...,Service,персоналу,244,253,positive
4761,16630,Уютная и тёплая домашняя обстановка! Милый и о...,Whole,ресторан,294,302,positive


В датасете очевидный дисбаланс классов -- положительных спанов аспектов значительно больше, чем остальных

In [ ]:
data['sentiment'].value_counts()

positive    3104
neutral      910
negative     660
both          89
Name: sentiment, dtype: int64

In [ ]:
cat_mapper = {cat: n for n, cat in enumerate(data.sentiment.unique())}
data.sentiment = data.sentiment.map(cat_mapper)

Приведем к формату @ слово @ текст для обучения. Не уверена, что это разумный и используемый в индустрии способ дообучения модели для решения задачи выделения тональности отдельного токена, но он сработал

In [ ]:
data['totrain'] = '@ ' + data['mention'] + ' @ ' + data['text']

In [ ]:
data

,review_id,text,category,mention,start,stop,sentiment,totrain
0,3976,"День 8-го марта прошёл, можно и итоги подвести...",Whole,ресторане,71,80,0,"@ ресторане @ День 8-го марта прошёл, можно и ..."
1,3976,"День 8-го марта прошёл, можно и итоги подвести...",Whole,ресторанах,198,208,0,"@ ресторанах @ День 8-го марта прошёл, можно и..."
2,3976,"День 8-го марта прошёл, можно и итоги подвести...",Whole,ресторане,256,265,0,"@ ресторане @ День 8-го марта прошёл, можно и ..."
3,3976,"День 8-го марта прошёл, можно и итоги подвести...",Service,Столик бронировали,267,285,0,"@ Столик бронировали @ День 8-го марта прошёл,..."
4,3976,"День 8-го марта прошёл, можно и итоги подвести...",Service,администратор,322,335,1,"@ администратор @ День 8-го марта прошёл, можн..."
...,...,...,...,...,...,...,...,...
4758,16630,Уютная и тёплая домашняя обстановка! Милый и о...,Service,обслуживание,85,97,1,@ обслуживание @ Уютная и тёплая домашняя обст...
4759,16630,Уютная и тёплая домашняя обстановка! Милый и о...,Food,Еда,99,102,1,@ Еда @ Уютная и тёплая домашняя обстановка! М...
4760,16630,Уютная и тёплая домашняя обстановка! Милый и о...,Service,персоналу,244,253,1,@ персоналу @ Уютная и тёплая домашняя обстано...
4761,16630,Уютная и тёплая домашняя обстановка! Милый и о...,Whole,ресторан,294,302,1,@ ресторан @ Уютная и тёплая домашняя обстанов...


In [ ]:
data['sentiment'].value_counts()

1    3104
0     910
3     660
2      89
Name: sentiment, dtype: int64

In [ ]:
sentences = data.totrain.values
labels = data.sentiment.values

# Загрузка модели

In [ ]:
from transformers import BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained("sberbank-ai/ruBert-base")

Loading BERT tokenizer...


vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

In [ ]:
from tqdm.auto import tqdm

In [ ]:

input_ids = []


for sent in tqdm(sentences, total=len(sentences)):
    encoded_sent = tokenizer.encode(
                        sent,
                        add_special_tokens = True,
                   )

    input_ids.append(encoded_sent)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

  0%|          | 0/4763 [00:00<?, ?it/s]

Original:  @ ресторане @ День 8-го марта прошёл, можно и итоги подвести. Решил написать отзыв о ресторане в котором отметили прекрасный весений праздник, прочитал отзывы edik077 и Rules77777и понял что либо мы были вразных ресторанах, либо у ребят что-то незаладилось. Но теперь о ресторане. Столик бронировали заранее и сделали так как предложил администратор т.е. сделали предварительный заказ, когда придя увидели полностью заполненый ресторан поняли что совет нам дали действительно правильный, в ресторане было человек 70-80, тут действительно горячее блюдо можно ждать весьма долго. Меню достаточно разнообразное и весьма вкусное, мне и моим друзьям понравилось всё что нам принесли, а принесли нам немало. Обслуживание может и не самое лучьшее в городе, но официанты делали всё что нужно. Должен сказать что ждать нам всёравно пришлось, не очень долго конечно, но всё-же. Интерьер хороший, удобные диваны.Очень хорошая музыкальная программа и весёлый ведущий. В общем я остался доволен своим в

In [ ]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  339


In [ ]:
MAX_LEN = max([len(sen) for sen in input_ids])

In [ ]:

from keras.preprocessing.sequence import pad_sequences

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long",
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 339 values...

Padding token: "[PAD]", ID: 0

Done.


In [ ]:

attention_masks = []

for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]

    attention_masks.append(att_mask)

In [ ]:
from sklearn.model_selection import train_test_split


train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,
                                                            random_state=2018, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

In [ ]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
model = BertForSequenceClassification.from_pretrained(
    "sberbank-ai/ruBert-base",
    num_labels = len(data.category.unique()),
    output_attentions = False,
    output_hidden_states = False,
)

model.cuda()

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruBert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
b = model.bert.pooler.dense.weight
c = model.classifier.weight
b = b.cpu().detach().numpy()
c = c.cpu().detach().numpy()

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (120138, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup
epochs = 4
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 100,
                                            num_training_steps = total_steps)


In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''

    elapsed_rounded = int(round((elapsed)))

    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import numpy as np

# Обучение

In [ ]:
import random
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


loss_values = []

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...').
    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()

        outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)
        loss = outputs[0]

        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)


    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():

            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy

        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    268.    Elapsed: 0:00:40.
  Batch    80  of    268.    Elapsed: 0:01:20.
  Batch   120  of    268.    Elapsed: 0:02:00.
  Batch   160  of    268.    Elapsed: 0:02:39.
  Batch   200  of    268.    Elapsed: 0:03:19.
  Batch   240  of    268.    Elapsed: 0:03:59.

  Average training loss: 0.92
  Training epcoh took: 0:04:27

Running Validation...
  Accuracy: 0.70
  Validation took: 0:00:10

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    268.    Elapsed: 0:00:40.
  Batch    80  of    268.    Elapsed: 0:01:20.
  Batch   120  of    268.    Elapsed: 0:01:59.
  Batch   160  of    268.    Elapsed: 0:02:39.
  Batch   200  of    268.    Elapsed: 0:03:19.
  Batch   240  of    268.    Elapsed: 0:03:59.

  Average training loss: 0.61
  Training epcoh took: 0:04:26

Running Validation...
  Accuracy: 0.81
  Validation took: 0:00:10

======== Epoch 3 / 4 ========
Training...
  Batch    40  of    268.    Elapsed: 0:00:40.
  

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model.save_pretrained('/content/gdrive/MyDrive/absa-project')